In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoTokenizer
from typing import List, Tuple

from sae_model import SparseAutoencoder
from activation_utils import get_llm_activations_residual_stream
from sae_trainer import extract_activations, create_data_loader, train_sparse_autoencoder

In [ ]:
# extract_activationsの変数設定
llm_model_name = "bert-base-uncased"    # 使用するLLMモデル名    
texts = [
    # "The quick brown fox jumps over the lazy dog.",
    # "A large language model can process and generate text.",
    # "Cats enjoy sleeping in warm, sunny spots.",
    # "This is a sample text for testing the Sparse Autoencoder.",
    # "PyTorch is a widely used deep learning framework." ,
    "A: Why are you carrying that wet umbrella? B: Because it started to rain on my way here, I had to use it.",
    "A: You seem very tired today. B: I am. I stayed up late to finish my report, so I didn't get much sleep.",
    "A: The room is so bright all of a sudden. What happened? B: I just pushed the main switch, which caused all the lights to turn on.",
    "A: I heard you passed the difficult exam. Congratulations! B: Thank you! I studied hard every day for it, so I'm glad my effort paid off."
    ]  # 使用するテキスト
target_layer_idx = 4    # 抽出するLLMの層インデックス
num_samples_for_training = 4  # 訓練に使用するサンプル数

# create_data_loaderの変数設定
batch_size = 256  # バッチサイズ

# train_sparse_autoencoderの変数設定
num_epochs = 200  # 訓練エポック数
sae_l1_coeff = 1 # スパース性の度合いを調整する係数

In [13]:
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModel.from_pretrained(llm_model_name)
training_texts = [texts[i] for i in range(num_samples_for_training)]

activations, activations_dict = get_llm_activations_residual_stream(
    llm_model, tokenizer, training_texts, target_layer_idx
)

In [14]:
data_loader = create_data_loader(activations, batch_size)
sae_model, training_losses, resonctruction_losses, sparsity_losses, sae_feature_dim, input_dim = train_sparse_autoencoder(
    activations, data_loader, num_epochs, sae_l1_coeff, llm_model_name=llm_model_name
)

Using device: mps
----------LLM・SAEの情報----------
LLMモデル名: bert-base-uncased
LLMの活性化ベクトルの次元数: 768, SAEの特徴次元: 844
----------------------------------

Starting SAE training for 200 epochs...
Epoch 1/200, Total Loss: 0.7702, Recon Loss: 0.5991, Sparse Loss: 0.1710
Epoch 2/200, Total Loss: 0.6589, Recon Loss: 0.5427, Sparse Loss: 0.1162
Epoch 3/200, Total Loss: 0.6003, Recon Loss: 0.5202, Sparse Loss: 0.0801
Epoch 4/200, Total Loss: 0.5656, Recon Loss: 0.5071, Sparse Loss: 0.0585
Epoch 5/200, Total Loss: 0.5423, Recon Loss: 0.4945, Sparse Loss: 0.0478
Epoch 6/200, Total Loss: 0.5232, Recon Loss: 0.4788, Sparse Loss: 0.0444
Epoch 7/200, Total Loss: 0.5055, Recon Loss: 0.4595, Sparse Loss: 0.0460
Epoch 8/200, Total Loss: 0.4889, Recon Loss: 0.4383, Sparse Loss: 0.0506
Epoch 9/200, Total Loss: 0.4736, Recon Loss: 0.4169, Sparse Loss: 0.0567
Epoch 10/200, Total Loss: 0.4586, Recon Loss: 0.3958, Sparse Loss: 0.0627
Epoch 11/200, Total Loss: 0.4434, Recon Loss: 0.3753, Sparse Loss: 0.0681
Epoch 1

In [15]:
activations_dict["A: Why are you carrying that wet umbrella? B: Because it started to rain on my way here, I had to use it."].shape

torch.Size([30, 768])

In [16]:
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
sae_model.to(device)
# sae_model.eval()

Using device: cpu


SparseAutoencoder(
  (encoder): Linear(in_features=768, out_features=844, bias=True)
  (relu): ReLU()
  (decoder): Linear(in_features=844, out_features=768, bias=True)
)

In [7]:
token_info_list = []
all_sae_features_list = []
global_token_idx = 0

# トークナイザーにpad_tokenが設定されているか確認 (活性化抽出時と条件を合わせるため)
if tokenizer.pad_token is None:
    if tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"トークナイザーのpad_tokenをeos_token ({tokenizer.pad_token}) に設定しました。")
    else:
        # これはデモスクリプト (demo_train_sae_gpt.py, demo_sae_train.py) や
        # activation_utils.py 内の処理と整合性を取る必要があります。
        print("警告: トークナイザーにpad_tokenが設定されていません。'[PAD]'を追加します。")
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        # LLMモデルの埋め込み層のリサイズが必要な場合がある点に注意 (model.resize_token_embeddings(len(tokenizer)))

if not activations_dict:
    print("活性化辞書 (activations_dict) が空です。特徴分析をスキップします。")
else:
   for original_text, token_activations in activations_dict.items():
      # 活性ベクトルをGPUに転送
      token_activations = token_activations.to(device)
      
      # 学習済みのSAEモデルを使用して、トークンの活性化をエンコード
      with torch.no_grad():
         sae_model_pre_relu = sae_model.encoder(token_activations)
         sae_features_for_text = sae_model.relu(sae_model_pre_relu)

      all_sae_features_list.append(sae_features_for_text.cpu())

      # トークンを取得
      inputs = tokenizer(original_text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
      
      # パディングトークンを除外するためにAttention Maskを使用
      attention_mask = inputs["attention_mask"].squeeze(0)
      input_ids_squeeze = inputs["input_ids"].squeeze(0)

      # トークンIDを取得
      actual_tokens_ids_for_text = input_ids_squeeze[attention_mask == 1]
      actual_tokens_str_list = tokenizer.convert_ids_to_tokens(actual_tokens_ids_for_text)

      if len(actual_tokens_str_list) != sae_features_for_text.shape[0]:
         print(f"Warning: Mismatch in token count for text: {original_text}")
         continue
      
      for token_idx_in_text in range(sae_features_for_text.shape[0]):
         token_info_list.append({
            "original_text": original_text,
            "token_idx_in_text": token_idx_in_text,   # テキスト内の実施あのトークンに対するインデックス
            "token_str": actual_tokens_str_list[token_idx_in_text],  # トークンの文字列表現
            "global_token_idx": global_token_idx,  # データセット全体を通したトークンのインデックス
         })
         
         global_token_idx += 1
         
# 以下結果の表示
if not all_sae_features_list:
   print("No SAE features found for the token.")

else:
   concatenated_sae_features = torch.cat(all_sae_features_list, dim=0)  # Shape: (num_tokens, sae_feature_dim)
   sae_total_features = concatenated_sae_features.shape[0]
   
   num_sae_features_to_analyze = min(10, sae_total_features) # 最大10個のSAE特徴を分析
   num_top_tokens_per_feature = 5  # 各SAE特徴に対して上位5つのトークンを分析

   # 指定した数のSAE特徴を分析するためのループ
   for feature_idx_to_analyze in range(num_sae_features_to_analyze):
      # 現在のSAE特徴次元に対応する前トークンの活性を取得
      feature_column_activation = concatenated_sae_features[:, feature_idx_to_analyze]
      
      # 上位k個の活性化とそのグローバルインデックスを取得
      actual_k = min(num_top_tokens_per_feature, len(feature_column_activation))
      if actual_k == 0 : continue
      
      top_k_values, top_k_global_indices = torch.topk(feature_column_activation, k=actual_k)
      
      print(f"\n--- SAE Feature {feature_idx_to_analyze} を最も強く活性化するトークン")
      
      if top_k_values.numel() == 0:
         print("No top tokens found for this feature.")
         continue
      
      # 上位k個のトークンの情報を表示
      for rank, (activation_value, global_token_idx_item) in enumerate(zip(top_k_values, top_k_global_indices)):
         global_idx = global_token_idx_item.item()    # テンソルから値を取り出す
         if global_idx < len(token_info_list):        
            token_info = token_info_list[global_idx]  # 取得したトークン情報
            
            text_snippet = token_info["original_text"]   # 元のテキスト
            
            # 文脈表示のために、元のテキストを再度トークナイズ(表示用)
            inputs_ctx = tokenizer(text_snippet,
                                 return_tensors="pt",
                                 truncation=True,
                                 max_length=128,
                                 padding="max_length",
                                 return_attention_mask=True)
            ids_ctx = inputs_ctx["input_ids"].squeeze()[inputs_ctx["attention_mask"].squeeze() == 1]
            tokens_ctx = tokenizer.convert_ids_to_tokens(ids_ctx)
            
            # 上記 tokens_ctx リスト内でのインデックスに相当
            tok_idx_in_ctx = token_info["token_idx_in_text"]
            
            context_window_size = 3    # 表示する前後のトークン数
            start_idx = max(0, tok_idx_in_ctx - context_window_size)
            end_idx = min(len(tokens_ctx), tok_idx_in_ctx + context_window_size + 1)
            
            context_display_parts = []                  
            for i in range(start_idx, end_idx):
               if i == tok_idx_in_ctx:
                  context_display_parts.append(f"**{tokens_ctx[i]}**")
               else:
                  context_display_parts.append(tokens_ctx[i])
            context_str = " ".join(context_display_parts)

            print(f"  順位 {rank + 1}: 活性化値 = {activation_value.item():.4f}")
            print(f"    トークン: '{token_info['token_str']}' (テキスト内の実トークンindex: {tok_idx_in_ctx})")
            print(f"    文脈: {context_str}")
            text_preview = (text_snippet[:70] + '...') if len(text_snippet) > 70 else text_snippet # テキストのプレビュー
            print(f"    元テキスト (一部): \"{text_preview}\"")
         else:
            print(f"  順位 {rank + 1}: エラー - グローバルインデックス {global_idx} が範囲外です。")                  
            


--- SAE Feature 0 を最も強く活性化するトークン
  順位 1: 活性化値 = 0.0000
    トークン: '[CLS]' (テキスト内の実トークンindex: 0)
    文脈: **[CLS]** the quick brown
    元テキスト (一部): "The quick brown fox jumps over the lazy dog."
  順位 2: 活性化値 = 0.0000
    トークン: 'the' (テキスト内の実トークンindex: 1)
    文脈: [CLS] **the** quick brown fox
    元テキスト (一部): "The quick brown fox jumps over the lazy dog."
  順位 3: 活性化値 = 0.0000
    トークン: 'quick' (テキスト内の実トークンindex: 2)
    文脈: [CLS] the **quick** brown fox jumps
    元テキスト (一部): "The quick brown fox jumps over the lazy dog."
  順位 4: 活性化値 = 0.0000
    トークン: 'brown' (テキスト内の実トークンindex: 3)
    文脈: [CLS] the quick **brown** fox jumps over
    元テキスト (一部): "The quick brown fox jumps over the lazy dog."
  順位 5: 活性化値 = 0.0000
    トークン: 'fox' (テキスト内の実トークンindex: 4)
    文脈: the quick brown **fox** jumps over the
    元テキスト (一部): "The quick brown fox jumps over the lazy dog."

--- SAE Feature 1 を最も強く活性化するトークン
  順位 1: 活性化値 = 0.0000
    トークン: '[CLS]' (テキスト内の実トークンindex: 0)
    文脈: **[CLS]** the quick brown
   